In [231]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
import time


class Init(Enum):
    MANUAL = 0
    RANDOM = 1


# Image number to be processed for task1
image_id = 2
# Number of quantized colors
k = 32
# Variable to decide initialization mode of color centers (manually or randomly)
initialization = Init.RANDOM
# Limit for the iterations i of the k-means algorithm
max_iterations = 100

In [232]:
def get_path_for_image(image_id):
    # Get directory of image files relative to this file's directory
    image_dir = os.path.join(os.getcwd(), 'data', 'images', 'task_1')
    # Get image path for the provided image id
    image_path = os.path.join(image_dir, 'image0{}.jpeg'.format(image_id))

    return image_path


def initialize_manually(image, k):
    pass

def initialize_randomly(image_matrix, k):
    # Create a numpy random number generator instance
    rng = np.random.default_rng(seed=537)

    # Generate k amount of random numbers (floats) between 0 and height of the image
    # Then, floor and convert to integer
    color_centers_heights = np.uint(np.floor(rng.uniform(low=0, high=image_matrix.shape[0], size=k)))
    # Generate k amount of random numbers (floats) between 0 and width of the image
    # Then, floor and convert to integer
    color_centers_widths = np.uint(np.floor(rng.uniform(low=0, high=image_matrix.shape[1], size=k)))

    # Stack height and width vectors column-wise into a matrix of shape [k, 2]
    # Each row representing a color center (x, y)
    color_centers = np.stack((color_centers_heights, color_centers_widths), axis=1)

    return color_centers

In [233]:
# Get path of image for given image id
image_path = get_path_for_image(image_id)
# Open the image using Pillow
image = Image.open(image_path)

# Convert the image to a numpy array of shape H x W x 3(RGB)
image_matrix = np.array(image)
# Create a new broadcast version of the image matrix of shape (H, W, 1, 3)
broadcast_image_matrix = image_matrix[:, :, np.newaxis, :].astype(dtype=np.int64)

# Initialize a cluster map with the same shape of the image where each
# coordinate (pixel) stores the index of the cluster it belongs to
initial_clusters = np.zeros(image_matrix.shape[0:2], image_matrix.dtype)

# Initialize color centers
if initialization == Init.MANUAL:
    initial_color_centers = initialize_manually(image, k)
else:
    initial_color_centers = initialize_randomly(image_matrix, k)
    
# Create variables to hold final results
final_clusters = None
final_color_centers = None

In [234]:
# Start the k-means algorithm
input_clusters = initial_clusters
input_color_centers = initial_color_centers
iterations = 0
total_time = 0
while True:
    iterations+=1
    start = time.time()
    
    # Retrieve RGB color vectors of input color centers of shape k x 3
    input_colors = image_matrix[input_color_centers[:, 0], input_color_centers[:, 1], :]
    # Create a new broadcast version of the input color matrix of shape (1, 1, k, 3)
    broadcast_input_colors = input_colors[np.newaxis, np.newaxis, :, :].astype(dtype=np.int64)
    
    # We need to calculate the distance of each pixel's color, to the colors of every color center.
    # Instead of doing this one pixel at a time with nested for loops, calculate as a whole via 
    # numpy vector operations (the reason why we created broadcast versions of the matrices)
    color_distances = np.sum(((broadcast_image_matrix - broadcast_input_colors)**2), axis=3)
    # Return the index of the minimum value along the distance axis (assign each pixel the
    # index of the closest color center)
    output_clusters = np.argmin(color_distances, axis=2)
    
    # Create a variable to store the new color centers based on new clusters
    output_color_centers = None
    
    for i in range(k):
        # Retrieve pixel coordinates for the current cluster index
        cluster_i_coordinates = np.argwhere(output_clusters == i)
        
        # Check if there are pixels assigned to the current cluster
        if len(cluster_i_coordinates) > 0:
            # Take the mean of the coordinates, floor and convert to integer
            new_ith_color_center = np.floor(np.mean(cluster_i_coordinates, axis=0)).astype(np.int64)
        else:
            # If the cluster is empty, keep the old color center
            new_ith_color_center = input_color_centers[i]
        
        
        if output_color_centers is None:
            output_color_centers = new_ith_color_center
        else:
            output_color_centers = np.vstack((output_color_centers, new_ith_color_center))
            
    if np.array_equal(output_clusters, input_clusters) or iterations >= max_iterations:
        final_clusters = output_clusters
        final_color_centers = output_color_centers
        
        end = time.time()
        round_time = (end - start) * 1000
        total_time += (end - start)
        print(f"Round {iterations}: {round_time:.2f} ms")
        print(f"Total time: {total_time:.2f} seconds")
        break
    else:
        input_clusters = output_clusters
        input_color_centers = output_color_centers
        
        end = time.time()
        round_time = (end - start) * 1000
        total_time += (end - start)
        print(f"Round {iterations}: {round_time:.2f} ms")
    

Round 1: 3029.04 ms
Round 2: 2515.00 ms
Round 3: 2424.57 ms
Round 4: 2390.11 ms
Round 5: 2394.93 ms
Round 6: 2419.87 ms
Round 7: 2838.75 ms
Round 8: 2421.51 ms
Round 9: 2406.39 ms
Round 10: 2378.48 ms
Round 11: 2368.83 ms
Round 12: 2447.60 ms
Round 13: 2361.26 ms
Round 14: 2360.48 ms
Round 15: 2319.87 ms
Round 16: 2306.97 ms
Round 17: 2301.78 ms
Round 18: 2348.10 ms
Round 19: 2308.45 ms
Round 20: 2331.17 ms
Round 21: 2389.36 ms
Round 22: 2348.68 ms
Round 23: 2326.43 ms
Round 24: 2292.04 ms
Round 25: 2315.05 ms
Round 26: 2297.10 ms
Round 27: 2542.40 ms
Round 28: 2775.05 ms
Round 29: 2566.90 ms
Round 30: 2466.79 ms
Round 31: 2466.69 ms
Round 32: 2297.22 ms
Round 33: 2296.25 ms
Round 34: 2300.99 ms
Round 35: 2338.92 ms
Round 36: 2325.08 ms
Round 37: 2332.12 ms
Round 38: 2371.54 ms
Round 39: 2281.61 ms
Round 40: 2279.12 ms
Round 41: 2479.11 ms
Round 42: 2391.74 ms
Round 43: 2420.39 ms
Round 44: 2725.08 ms
Round 45: 2675.19 ms
Round 46: 2499.85 ms
Round 47: 2502.25 ms
Round 48: 2586.65 ms
R

In [235]:
print(final_clusters.shape)
print(final_clusters)

(1200, 1920)
[[ 9  9  9 ...  6  6  6]
 [ 9 12  9 ...  6  6  6]
 [ 9  9  9 ...  6  6  6]
 ...
 [15 15 15 ... 15  2  2]
 [15 15 15 ... 15  2  2]
 [15 15 15 ... 15  2  2]]


In [236]:
print(final_color_centers.shape)
print(final_color_centers)

(32, 2)
[[ 426 1001]
 [ 606  884]
 [ 518  982]
 [ 810  608]
 [ 424 1002]
 [ 507  916]
 [ 599  941]
 [ 375 1043]
 [ 531  883]
 [ 481  844]
 [ 476  995]
 [ 401 1220]
 [ 536  901]
 [ 591  942]
 [ 453 1047]
 [ 739  832]
 [ 482  957]
 [ 405 1063]
 [ 742  642]
 [ 461 1011]
 [ 504 1050]
 [ 764  672]
 [ 700  836]
 [ 662  979]
 [ 395 1149]
 [ 492  925]
 [ 673 1568]
 [ 434 1098]
 [ 353 1239]
 [ 470  747]
 [ 463 1291]
 [ 800  550]]


In [237]:
final_colors = image_matrix[final_color_centers[:, 0], final_color_centers[:, 1], :]
print(final_colors.shape)
print(final_colors)

(32, 3)
[[ 33 100 152]
 [  7  34 103]
 [ 23 117 171]
 [  3   8  30]
 [ 64 147 189]
 [100 222 255]
 [ 11  64 180]
 [ 48 155 211]
 [ 45 167 234]
 [  8  50  90]
 [ 42 106 134]
 [  4   0  57]
 [ 82 198 221]
 [  7  55 166]
 [  0   0   4]
 [  4  24  51]
 [  0   2  14]
 [  2   2   4]
 [  2  14  72]
 [  2  12  48]
 [  0  97 204]
 [  0  22  98]
 [ 47 183 235]
 [ 27 108 223]
 [  7   1   5]
 [ 13  36  86]
 [  1  55   6]
 [  5   2   9]
 [  0   7   5]
 [ 22  98 184]
 [ 29 105 201]
 [  3  22 124]]


In [238]:
quantized_image_matrix = np.zeros((image_matrix.shape[0], image_matrix.shape[1], 3), dtype=np.uint8)
print(quantized_image_matrix.shape)
print(quantized_image_matrix)

(1200, 1920, 3)
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [239]:
quantized_image_matrix = final_colors[final_clusters]
print(quantized_image_matrix.shape)
print(quantized_image_matrix)

(1200, 1920, 3)
[[[  8  50  90]
  [  8  50  90]
  [  8  50  90]
  ...
  [ 11  64 180]
  [ 11  64 180]
  [ 11  64 180]]

 [[  8  50  90]
  [ 82 198 221]
  [  8  50  90]
  ...
  [ 11  64 180]
  [ 11  64 180]
  [ 11  64 180]]

 [[  8  50  90]
  [  8  50  90]
  [  8  50  90]
  ...
  [ 11  64 180]
  [ 11  64 180]
  [ 11  64 180]]

 ...

 [[  4  24  51]
  [  4  24  51]
  [  4  24  51]
  ...
  [  4  24  51]
  [ 23 117 171]
  [ 23 117 171]]

 [[  4  24  51]
  [  4  24  51]
  [  4  24  51]
  ...
  [  4  24  51]
  [ 23 117 171]
  [ 23 117 171]]

 [[  4  24  51]
  [  4  24  51]
  [  4  24  51]
  ...
  [  4  24  51]
  [ 23 117 171]
  [ 23 117 171]]]


In [240]:
# Convert the final quantized image matrix to an image
quantized_image = Image.fromarray(quantized_image_matrix.astype(np.uint8))

# Display the quantized image
quantized_image.show()

In [201]:
print(initial_color_centers.shape)
print(initial_color_centers)

(32, 2)
[[ 428  871]
 [1034 1856]
 [ 385  462]
 [ 622 1695]
 [ 912  760]
 [ 520  965]
 [ 176   94]
 [ 571  463]
 [ 370  355]
 [ 430 1740]
 [ 734 1262]
 [ 235 1780]
 [ 186  277]
 [ 940 1695]
 [ 105  805]
 [ 807 1468]
 [ 118  133]
 [1103  801]
 [ 499  425]
 [  40 1657]
 [1002  767]
 [  16 1493]
 [   7 1130]
 [ 709 1094]
 [ 147 1047]
 [ 583  202]
 [ 245 1489]
 [ 546 1308]
 [ 164  309]
 [ 777 1833]
 [ 800  975]
 [ 631 1028]]


In [156]:
image_matrix.dtype

dtype('uint8')

In [131]:
# Initialize a cluster map with the same shape of the image where each
# coordinate (pixel) stores the index of the cluster it belongs to
initial_clusters = np.zeros(image_matrix.shape[0:2], image_matrix.dtype)

In [133]:
final_clusters = None
final_centers = None

In [134]:
input_clusters = initial_clusters
input_color_centers = initial_color_centers

In [140]:
input_colors = image_matrix[input_color_centers[:, 0], input_color_centers[:, 1], :]

In [175]:
print(input_colors.shape)
print(input_colors)

(8, 3)
[[  5  34  12]
 [  2  74  70]
 [  9  71 148]
 [  3  20   4]
 [  4  69 187]
 [106 209  66]
 [  1 167  41]
 [ 10 173 116]]


In [144]:
image_matrix.shape

(1200, 1920, 3)

In [145]:
input_colors.shape

(8, 3)

In [176]:
broadcast_image_matrix = image_matrix[:, :, np.newaxis, :].astype(dtype=np.int64)
print(broadcast_image_matrix.shape)
print(broadcast_image_matrix)

(1200, 1920, 1, 3)
[[[[  1 130 110]]

  [[  0 126 106]]

  [[  2 129 110]]

  ...

  [[  3  38   5]]

  [[  3  39   3]]

  [[ 10  46  10]]]


 [[[  0 123 104]]

  [[  0 119 100]]

  [[  0 123 105]]

  ...

  [[  3  38   5]]

  [[  3  39   3]]

  [[ 10  46  10]]]


 [[[  0 123 103]]

  [[  0 120 100]]

  [[  4 125 106]]

  ...

  [[  3  41   4]]

  [[  3  41   4]]

  [[ 10  48  11]]]


 ...


 [[[  7  19   7]]

  [[  1  13   1]]

  [[  2  14   2]]

  ...

  [[  1   2   4]]

  [[  3   4   6]]

  [[  6   7   9]]]


 [[[  7  19   7]]

  [[  1  13   1]]

  [[  2  14   2]]

  ...

  [[  0   1   3]]

  [[  2   3   5]]

  [[  6   7   9]]]


 [[[  7  19   7]]

  [[  1  13   1]]

  [[  3  15   3]]

  ...

  [[  0   1   3]]

  [[  2   3   5]]

  [[  5   6   8]]]]


In [178]:
broadcast_input_colors = input_colors[np.newaxis, np.newaxis, :, :].astype(dtype=np.int64)
print(broadcast_input_colors.shape)
print(broadcast_input_colors)

(1, 1, 8, 3)
[[[[  5  34  12]
   [  2  74  70]
   [  9  71 148]
   [  3  20   4]
   [  4  69 187]
   [106 209  66]
   [  1 167  41]
   [ 10 173 116]]]]


In [179]:
color_distances = broadcast_image_matrix - broadcast_input_colors
print(color_distances.shape)
print(color_distances)

(1200, 1920, 8, 3)
[[[[  -4   96   98]
   [  -1   56   40]
   [  -8   59  -38]
   ...
   [-105  -79   44]
   [   0  -37   69]
   [  -9  -43   -6]]

  [[  -5   92   94]
   [  -2   52   36]
   [  -9   55  -42]
   ...
   [-106  -83   40]
   [  -1  -41   65]
   [ -10  -47  -10]]

  [[  -3   95   98]
   [   0   55   40]
   [  -7   58  -38]
   ...
   [-104  -80   44]
   [   1  -38   69]
   [  -8  -44   -6]]

  ...

  [[  -2    4   -7]
   [   1  -36  -65]
   [  -6  -33 -143]
   ...
   [-103 -171  -61]
   [   2 -129  -36]
   [  -7 -135 -111]]

  [[  -2    5   -9]
   [   1  -35  -67]
   [  -6  -32 -145]
   ...
   [-103 -170  -63]
   [   2 -128  -38]
   [  -7 -134 -113]]

  [[   5   12   -2]
   [   8  -28  -60]
   [   1  -25 -138]
   ...
   [ -96 -163  -56]
   [   9 -121  -31]
   [   0 -127 -106]]]


 [[[  -5   89   92]
   [  -2   49   34]
   [  -9   52  -44]
   ...
   [-106  -86   38]
   [  -1  -44   63]
   [ -10  -50  -12]]

  [[  -5   85   88]
   [  -2   45   30]
   [  -9   48  -48]
   ...
  

In [180]:
color_distances = color_distances**2
print(color_distances.shape)
print(color_distances)

(1200, 1920, 8, 3)
[[[[   16  9216  9604]
   [    1  3136  1600]
   [   64  3481  1444]
   ...
   [11025  6241  1936]
   [    0  1369  4761]
   [   81  1849    36]]

  [[   25  8464  8836]
   [    4  2704  1296]
   [   81  3025  1764]
   ...
   [11236  6889  1600]
   [    1  1681  4225]
   [  100  2209   100]]

  [[    9  9025  9604]
   [    0  3025  1600]
   [   49  3364  1444]
   ...
   [10816  6400  1936]
   [    1  1444  4761]
   [   64  1936    36]]

  ...

  [[    4    16    49]
   [    1  1296  4225]
   [   36  1089 20449]
   ...
   [10609 29241  3721]
   [    4 16641  1296]
   [   49 18225 12321]]

  [[    4    25    81]
   [    1  1225  4489]
   [   36  1024 21025]
   ...
   [10609 28900  3969]
   [    4 16384  1444]
   [   49 17956 12769]]

  [[   25   144     4]
   [   64   784  3600]
   [    1   625 19044]
   ...
   [ 9216 26569  3136]
   [   81 14641   961]
   [    0 16129 11236]]]


 [[[   25  7921  8464]
   [    4  2401  1156]
   [   81  2704  1936]
   ...
   [11236  739

In [181]:
color_distances = np.sum(color_distances, axis=3)
print(color_distances.shape)
print(color_distances)

(1200, 1920, 8)
[[[18836  4737  4989 ... 19202  6130  1966]
  [17325  4004  4870 ... 19725  5907  2409]
  [18638  4625  4857 ... 19152  6206  2036]
  ...
  [   69  5522 21574 ... 43571 17941 30595]
  [  110  5715 22085 ... 43478 17832 30774]
  [  173  4448 19670 ... 38921 15683 27365]]

 [[16410  3561  4721 ... 20076  5906  2744]
  [14994  2929  4689 ... 20492  5786  3272]
  [16595  3630  4634 ... 20153  6033  2721]
  ...
  [   69  5522 21574 ... 43571 17941 30595]
  [  110  5715 22085 ... 43478 17832 30774]
  [  173  4448 19670 ... 38921 15683 27365]]

 [[16227  3494  4810 ... 20001  5781  2769]
  [15165  3020  4786 ... 20313  5691  3165]
  [17118  3901  4705 ... 19060  5998  2440]
  ...
  [  117  5446 21672 ... 42677 17249 30017]
  [  117  5446 21672 ... 42677 17249 30017]
  [  222  4221 19299 ... 38162 15142 26650]]

 ...

 [[  254  7019 22589 ... 49382 23096 35606]
  [  578  8483 25037 ... 53666 25316 38906]
  [  509  8224 24614 ... 52937 24931 38341]
  ...
  [ 1104  9541 25561 ...

In [190]:
print(color_distances[0, 1917, :])

[   69  5522 21574   325 34086 43571 17941 30595]


In [182]:
min_distances = np.argmin(color_distances, axis=2)
print(min_distances.shape)
print(min_distances)

(1200, 1920)
[[7 7 7 ... 0 0 0]
 [7 1 7 ... 0 0 0]
 [7 1 7 ... 0 0 0]
 ...
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]]


In [191]:
cluster_zero_coordinates = np.argwhere(min_distances == 0)
print(cluster_zero_coordinates.shape)
print(cluster_zero_coordinates)

(406464, 2)
[[   0   46]
 [   0   47]
 [   0   48]
 ...
 [1199  959]
 [1199 1302]
 [1199 1303]]


In [196]:
new_cluster_zero_center = np.floor(np.mean(cluster_zero_coordinates, axis=0)).astype(np.int64)
print(new_cluster_zero_center.shape)
print(new_cluster_zero_center)

(2,)
[ 633 1051]


In [212]:
cluster_3_coordinates = np.argwhere(min_distances == 3)

In [213]:
cluster_3_coordinates

array([[   0,   97],
       [   0,   98],
       [   0,   99],
       ...,
       [1199, 1917],
       [1199, 1918],
       [1199, 1919]])

In [209]:
new_cluster_centers = None

for i in range(8):
    cluster_i_coordinates = np.argwhere(min_distances == i)
    new_cluster_i_center = np.floor(np.mean(cluster_i_coordinates, axis=0)).astype(np.int64)
    if new_cluster_centers is None:
        new_cluster_centers = new_cluster_i_center
    else:
        new_cluster_centers = np.vstack((new_cluster_centers, new_cluster_i_center))
    
print(new_cluster_centers.shape)
print(new_cluster_centers)

(8, 2)
[[ 633 1051]
 [ 503  766]
 [ 678  765]
 [ 670  872]
 [ 597  955]
 [ 670 1517]
 [ 538  824]
 [ 451  966]]


In [173]:
np.argmin(np.sum(one**2, axis=-1), axis=-1, keepdims=True)

array([[[7],
        [7],
        [7],
        ...,
        [0],
        [0],
        [0]],

       [[7],
        [1],
        [7],
        ...,
        [0],
        [0],
        [0]],

       [[7],
        [1],
        [7],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[3],
        [3],
        [3],
        ...,
        [3],
        [3],
        [3]],

       [[3],
        [3],
        [3],
        ...,
        [3],
        [3],
        [3]],

       [[3],
        [3],
        [3],
        ...,
        [3],
        [3],
        [3]]])